In [1]:
from __future__ import print_function, division
from scipy.optimize import fsolve #to get the initial value of vecp0
import numpy as np
from scikits.odes import dae
import pandas as pd
import time
import cantera as ct
import os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import prettyplotlib as ppl
from prettyplotlib import brewer2mpl
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['font.family'] = 'Times New Roman'
colors = brewer2mpl.get_map('Set2','qualitative',8).mpl_colors

To implement the 1D plug flow reactor model with surface chemistry, this code is tested for the simplest case -- isothermal tube with laminar flow. The governing equations and constraints are listed as follows
\begin{align}
    u\frac{d\rho}{dz} + \rho\frac{du}{dz} - \frac{p'}{A_c}\sum^{K_g}\dot{s_k}W_k &= 0\\
    \rho u A_c\frac{dY_k}{dz} + Y_k p'\sum^{K_g}\dot{s_k}W_k - \dot{\omega_k}W_kA_c - \dot{s_k}W_k p' &= 0\\
    2\rho u \frac{du}{dz} + u^2\frac{d\rho}{dz} + \frac{dP}{dz} + \frac{8\rho u \nu \pi}{A_c} &= 0 \\
    P\bar{W} &= \rho RT\\
    \frac{dZ_k}{dz} &= \frac{\dot{s_k}\sigma_k}{\Gamma u}                                                 
\end{align}
First step is to determine the initial value of u, $\rho$, $Y_k$, P, $Z_k$ and their first derivative at the entrance of the tube:                                        

## 1. Calculate the initial value of Z_k at the entrance of the tube

In [2]:
mech ='/Users/yuanjie/Dropbox/Cantera_intern/code/gas_surf_phase/gas_surf_bulk/mec.cti'
#mech = '/Users/yuanjie/Dropbox/Cantera_intern/code/gas_surf_phase/g_s_b/Sandia_mec.cti'
#import the models for gas and bulk
ct.suppress_thermo_warnings()
gas, bulk_Si, bulk_N = ct.import_phases(mech,['gas','SiBulk','NBulk',])

#import the model for gas-Si-N interface
gas_Si_N_interface = ct.Interface(mech, 'SI3N4',[gas,bulk_Si,bulk_N])
T = 1713 #K
#p = 2 * ct.one_atm / 760.0 #Pa ~2Torr
p = 2.632E-3*ct.one_atm 
gas.TPX = T,p,"NH3:6, SiF4:1"
bulk_Si.TP = T,p
bulk_N.TP = T,p
gas_Si_N_interface.TP = T,p
gas_Si_N_interface.advance_coverages(50.0)

clf = []
for i in range(gas_Si_N_interface.n_species):
    spec_name = gas_Si_N_interface.species_names[i]
    sdot_k = gas_Si_N_interface.net_production_rates[gas_Si_N_interface.kinetics_species_index(spec_name)]
    Z_k = gas_Si_N_interface.coverages[i]
    clf.append((spec_name,sdot_k,Z_k))
"""
Paper's results
HN_SIF(S)   = 6.242E-02
H2NFSINH(S) = 2.411E-04
F3SI_NH2(S) = 3.136E-04  
HN(FSINH)2(S)= 4.821E-04
F2SINH(S) = 2.081E-02
HN_NH2(S) = 9.157E-01
"""
clf

[('HN_SIF(S)', -1.0587911840678754e-22, 0.062570074435456077),
 ('HN_NH2(S)', 1.0587911840678754e-22, 0.91554147625557647),
 ('F3SI_NH2(S)', -1.0587911840678754e-22, 0.00031421819923261961),
 ('F2SINH(S)', 0.0, 0.020851175294545006),
 ('H2NFSINH(S)', 7.9409338805090657e-23, 0.0002410186050631289),
 ('HN(FSINH)2(S)', 0.0, 0.0004820372101262578)]

## 2. Solve DAE system
### 2.1 Calculate the initial values of vec0 and vecp0

In [3]:
%%latex
Since IDA solver needs input the initial value of variables and their primes. The following nonlinear equation system has been solved by the nonlinear solver.
\begin{align}
    u_0\rho_0' + \rho_0 u_0' - \frac{p'}{A_c}\sum^{K_g}\dot{s_k}W_k &= 0\\
    \rho_0 u_0 A_c Y_{k,0}' + Y_{k,0} p'\sum^{K_g}\dot{s_k}W_k - \dot{\omega_k}W_kA_c - \dot{s_k}W_k p' &=0 \\                 
    2\rho_0 u_0 u_0' + u_0^2\rho_0' + P_0' + \frac{8\rho_0 \nu \pi}{A_c} &=0\\
    -RT\rho_0' + \bar{W_0}P_0' - P_0\frac{\sum^{K_g}Y_k'/W_k}{(\sum^{K_g}Y_k/W_k)^2} &= 0 \\
    Z_{k,0}' - \frac{\dot{s_k}\sigma_k}{\Gamma u_0} &=0
\end{align}

<IPython.core.display.Latex object>

In [4]:
#solve the nonlinear equation
R =  8.314 #J / mol. K
T = 1713
D = 5.08 * 10**-2 #diameter of the tube [m]
Ac = np.pi * D**2/4 # cross section of the tube [m]
Vdot = 588 * 10**-6/60 # volumetric rate [m^3/s] 
u0 = Vdot * Ac
nu = 5.7E-5 #kg/(m s) viscosity
Gamma = 0.417E-7 #mol/m^2 site desity
rho0 = gas.density

#p = 2 * ct.one_atm / 760.0 #Pa ~2Torr
p0 = 2.632E-3 * ct.one_atm
Z_k_0 = [float(i) for i in np.array(clf)[:,2]] #Z_HN_SIF_0,Z_HN_NH2_0,Z_F3SI_NH2_0,Z_F2SINH_0,Z_H2NFSINH_0,Z_HNFSINH2_0 site fraction for surface species
sdot_k_0 = [float(i) for i in np.array(clf)[:,1]]
sigma_k = [2, 2, 2, 2, 2, 4] #site occupancy number
perim = np.pi * D #perimeter of the tube

def equations(vecp0):
    eq = np.empty((26))
    eq[0] = vecp0[0]*rho0 + vecp0[1]*u0 - perim * np.sum(gas_Si_N_interface.net_production_rates[:gas.n_species] * gas.molecular_weights)/Ac
    for i in range(gas.n_species):   
        eq[1+i] = rho0 * u0 * Ac * vecp0[2+i]\
                  + gas.Y[i] * perim * np.sum(gas_Si_N_interface.net_production_rates[:gas.n_species] * gas.molecular_weights)\
                  - gas.net_production_rates[i] * gas.molecular_weights[i] * Ac\
                  - gas_Si_N_interface.net_production_rates[i] * gas.molecular_weights[i] * perim
    eq[1+gas.n_species] = 2*rho0 * u0 * vecp0[0] + u0**2*vecp0[1] + vecp0[2+gas.n_species] + 8*rho0*u0*nu*np.pi/Ac
    eq[2+gas.n_species] = -R*T*vecp0[1] + gas.mean_molecular_weight*vecp0[2+gas.n_species] - p0*1/np.power(np.sum(gas.Y/gas.molecular_weights),2)*np.sum(vecp0[2:2+gas.n_species]/gas.molecular_weights)
    for j in range(gas_Si_N_interface.n_species):
        eq[3+gas.n_species+j] = vecp0[3+gas.n_species+j] - gas_Si_N_interface.net_production_rates[gas.n_species+j]*sigma_k[j]/(Gamma*u0)
    return eq
vec0 = np.append(np.append(np.append([u0, rho0], gas.Y),p0),Z_k_0)
vecp_guess = np.array([(11.98 - 11.53)/0.1, #du/dz|z=0
                       (5.124E-4 - 5.515E-4)/0.1, #d\rho/dz|z=0
                       4.982E-6/0.1, #dY_H2/dz|z=0
                       2.26E-8/0.1, #dY_H/dz|z=0
                       1.305E-9/0.1, #dY_N2/dz|z=0
                       1.304E-13/0.1,#dY_N/dz|z=0
                       1.27E-9/0.1,#dY_NH/dz|z=0
                       9.969E-6/0.1,#dY_NH2/dz|z=0
                       5.472E-11/0.1,#dY_NNH/dz|z=0
                       1.441E-9/0.1,#dY_N2H2/dz|z=0
                       4.683E-11/0.1,#dY_N2H3/dz|z=0
                       1.786E-12/0.1,#dY_N2H4/dz|z=0
                       8.362E-2/0.1,#dY_HF/dz|z=0
                       4.476E-11/0.1,#dY_F/dz|z=0
                       (1.168E-1 - 1.427E-1)/0.1,#dY_SIF4/dz|z=0
                       1.376E-10/0.1,#dY_SIF3/dz|z=0
                       7.613E-11/0.1,#dY_SIHF3/dz|z=0
                       3.9E-10/0.1,#dY_SIF3NH2/dz|z=0
                       (7.995E-1 - 8.573E-1)/0.1,#dY_NH3/dz|z=0
                       (1.994 - 2)* ct.one_atm / 760.0, #dp/dz|z=0
                       (5.536E-2 - 6.242E-2)/0.1, #dZ_HN_SIF/dz|z=0
                       (2.586E-4 - 3.136E-4)/0.1, #dZ_F3SI_NH2/dz|z=0
                       (1.845E-2 - 2.081E-2)/0.1, #dZ_F2SINH/dz|z=0
                       (2.241E-4 - 2.411E-4)/0.1, #dZ_F2SINH/dz|z=0
                       (4.482E-4 - 4.821E-4)/0.1, #dZ_HNFSINH/dz|z=0
                       (9.253E-1 - 9.157E-1)/0.1]) #dZ_HN_NH2/dz|z=0 
vecp0= fsolve (equations, vecp_guess)  

/Users/yuanjie/anaconda/lib/python2.7/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


### 2.2 IDA solver

In [5]:
def residual(z, vec, vecp, result):
    """ we create the residual equations for the problem"""
    T = 1713
    gas.TPY = T,vec[2+gas.n_species],vec[2:2+gas.n_species]
    bulk_Si.TP = T,vec[2+gas.n_species]
    bulk_N.TP = T,vec[2+gas.n_species]
    gas_Si_N_interface.TP = T,vec[2+gas.n_species]
    gas_Si_N_interface.advance_coverages(50.0)
    sdot_gas = []
    for i in range(gas.n_species):
        spec_name = gas.species_names[i]
        sdot_k_gas = gas_Si_N_interface.net_production_rates[gas_Si_N_interface.kinetics_species_index(spec_name)]
        sdot_gas.append(sdot_k_gas)
    result[0] = vec[0]*vecp[1]+vec[1]*vecp[0]-perim*np.sum(sdot_gas*gas.molecular_weights)/Ac
    for k in range(gas.n_species):
        result[1+k] = vec[1]*vec[0]*Ac*vecp[2+k] + vec[2+k]*perim*np.sum(sdot_gas*gas.molecular_weights)\
                      - gas.net_production_rates[k]*gas.molecular_weights[k]*Ac\
                      - gas_Si_N_interface.net_production_rates[k]*gas.molecular_weights[k]*perim
    result[1+gas.n_species] = 2*vec[1]*vec[0]*vecp[0]+vec[0]**2*vecp[1]+vecp[2+gas.n_species]+8*vec[1]*vec[0]*nu*np.pi/Ac
    result[2+gas.n_species] = vec[2+gas.n_species]*gas.mean_molecular_weight-vec[1]*R*T
    for j in range(gas_Si_N_interface.n_species):
        result[3+gas.n_species+j] = vecp[3+gas.n_species+j]\
        - gas_Si_N_interface.net_production_rates[gas.n_species+j]*sigma_k[j]/(Gamma*vec[0])

In [6]:
solver = dae('ida', residual, 
             compute_initcond='yp0', #If yp0, then the differential variables (y of the ode system at time 0) will be used to solve for the derivatives of the differential variables, so yp0 will be calculated
             first_step_size=1e-18,
             atol=1e-6, #absolute tolerance for solution
             rtol=1e-6, #relative tolerance for solution
             algebraic_vars_idx=[4], #If the given problem is of type DAE, some items of the residual
                    #vector returned by the 'resfn' have to be treated as
                    #algebraic equations, and algebraic variables must be defined.
                    #These algebraic variables are denoted by the position (index)
                    #in the state vector y.
                    #All these indexes have to be specified in the
                    #'algebraic_vars_idx' array.
             #compute_initcond_t0 = 60,#When calculating the initial condition, specifies the time
                                      # until which the solver tries to
                                      #get the consistent values for either y0 or yp0 relative to
                                      #the starting time. Positive if t1 > t0, negative if t1 < t0
             max_steps=50000,
             old_api=False)#Forces use of old api (tuple of 7) if True or
                    #new api (namedtuple) if False.
                    #Other options may require new api, hence using this should
                    #be avoided if possible.
solution = solver.solve(np.arange(0,0.6,0.1), vec0, vecp0)
solution

SolverReturn(flag=-13, values=SolverVariables(t=None, y=None, ydot=None), errors=SolverVariables(t=0.0, y=array([  1.98629332e-08,   5.51738330e-04,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   5.04595486e-01,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         4.95404514e-01,   2.66687400e+02,   6.25700744e-02,
         9.15541476e-01,   3.14218199e-04,   2.08511753e-02,
         2.41018605e-04,   4.82037210e-04]), ydot=array([  4.49999682e+00,   2.31245815e+00,  -7.06645189e+00,
        -1.61091844e+00,   1.12174324e-07,   1.30399953e-12,
         6.09329088e-08,   2.27943778e+00,   3.21752467e-04,
         1.23864522e-07,   4.68299837e-10,   1.78599936e-11,
         9.72829940e+01,  -3.49086163e-02,  -2.58999635e-01,
        -1.79701857e+00,   3.09373344e-09,   3.8999